In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import numpy as np
import pandas as pd
import os
import pickle
import gc
#分布確認
import pandas_profiling as pdp
#可視化
import matplotlib.pyplot as plt
#前処理
from sklearn.preprocessing import StandardScaler, MinMaxScaler,LabelEncoder, OneHotEncoder
#モデリング
from sklearn.model_selection import train_test_split, KFold,StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")

In [3]:
df_train = pd.read_csv("../input/titanic/train.csv")
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
print(df_train.shape)
print("レコード数：", len(df_train))
print("カラム", len(df_train.columns))

(891, 12)
レコード数： 891
カラム 12


In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
df_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
x_train, y_train, id_train = df_train[["Pclass", "Fare"]], \
df_train[["Survived"]], \
df_train[["PassengerId"]]
print(x_train.shape, y_train.shape, id_train.shape)

(891, 2) (891, 1) (891, 1)


In [8]:
x_tr, x_va, y_tr, y_va = train_test_split(x_train,
                                          y_train,test_size = 0.2,
                                          shuffle = True,
                                          stratify = y_train,
                                          random_state = 123)
print(x_tr.shape, y_tr.shape)
print(x_va.shape, y_va.shape)
print("y_train:{:.3f}, y_tr:{:.3f}, y_va:{:.3f}".format(
y_train["Survived"].mean(),
y_tr["Survived"].mean(),
y_va["Survived"].mean(),
))

(712, 2) (712, 1)
(179, 2) (179, 1)
y_train:0.384, y_tr:0.383, y_va:0.385


In [9]:
n_splits = 5
cv = list(StratifiedKFold(n_splits = n_splits,
                         shuffle = True,
                         random_state = 123).split(x_train, y_train))

for nfold in np.arange(n_splits):
    print("-" * 20, nfold, "-" * 20)
    idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
    x_tr, y_tr = x_train.loc[idx_tr, :], y_train.loc[idx_tr, :]
    x_va, y_va = x_train.loc[idx_va, :], y_train.loc[idx_va, :]
    print(x_tr.shape, y_tr.shape)
    print(x_va.shape, y_va.shape)
    print("y_train:{:.3f}, y_tr:{:.3f}, y_va:{:.3f}".format(
        y_train["Survived"].mean(),
        y_tr["Survived"].mean(),
        y_va["Survived"].mean(),
        ))

-------------------- 0 --------------------
(712, 2) (712, 1)
(179, 2) (179, 1)
y_train:0.384, y_tr:0.383, y_va:0.385
-------------------- 1 --------------------
(713, 2) (713, 1)
(178, 2) (178, 1)
y_train:0.384, y_tr:0.384, y_va:0.382
-------------------- 2 --------------------
(713, 2) (713, 1)
(178, 2) (178, 1)
y_train:0.384, y_tr:0.384, y_va:0.382
-------------------- 3 --------------------
(713, 2) (713, 1)
(178, 2) (178, 1)
y_train:0.384, y_tr:0.384, y_va:0.382
-------------------- 4 --------------------
(713, 2) (713, 1)
(178, 2) (178, 1)
y_train:0.384, y_tr:0.383, y_va:0.388


In [10]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.1,
    'num_leaves': 16,
    'n_estimators': 100000,
    "random_state": 123,
    "importance_type": "gain",
}

model = lgb.LGBMClassifier(**params)
model.fit(x_tr, y_tr, eval_set = [(x_tr, y_tr), (x_va, y_va)], early_stopping_rounds = 100, verbose = 10,)

[10]	training's auc: 0.775008	valid_1's auc: 0.745911
[20]	training's auc: 0.795725	valid_1's auc: 0.749036
[30]	training's auc: 0.804937	valid_1's auc: 0.751828
[40]	training's auc: 0.811443	valid_1's auc: 0.749036
[50]	training's auc: 0.817045	valid_1's auc: 0.747972
[60]	training's auc: 0.821208	valid_1's auc: 0.745778
[70]	training's auc: 0.825062	valid_1's auc: 0.740659
[80]	training's auc: 0.829433	valid_1's auc: 0.739596
[90]	training's auc: 0.832413	valid_1's auc: 0.738532
[100]	training's auc: 0.835177	valid_1's auc: 0.735607


LGBMClassifier(importance_type='gain', metric='auc', n_estimators=100000,
               num_leaves=16, objective='binary', random_state=123)

In [11]:
y_tr_pred = model.predict(x_tr)
y_va_pred = model.predict(x_va)
metric_tr = accuracy_score(y_tr, y_tr_pred)
metric_va = accuracy_score(y_va, y_va_pred)
print("[accuracy] tr: {:.2f}, va: {:.2f}".format(metric_tr, metric_va))

[accuracy] tr: 0.62, va: 0.61


In [12]:
imp = pd.DataFrame({"col":x_train.columns, "imp":model.feature_importances_})
imp.sort_values("imp", ascending = False, ignore_index = True)

,col,imp
0,Fare,175.473267
1,Pclass,169.153896


In [13]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.1,
    'num_leaves': 16,
    'n_estimators': 100000,
    "random_state": 123,
    "importance_type": "gain",
}

metrics = []
imp = pd.DataFrame()

n_splits = 5
cv = list(StratifiedKFold(n_splits = n_splits,
                         shuffle = True,
                         random_state = 123).split(x_train, y_train))

for nfold in np.arange(n_splits):
    print("-" * 20, nfold, "-" * 20)
    idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
    x_tr, y_tr = x_train.loc[idx_tr, :], y_train.loc[idx_tr, :]
    x_va, y_va = x_train.loc[idx_va, :], y_train.loc[idx_va, :]
    print(x_tr.shape, y_tr.shape)
    print(x_va.shape, y_va.shape)
    print("y_train:{:.3f}, y_tr:{:.3f}, y_va:{:.3f}".format(
        y_train["Survived"].mean(),
        y_tr["Survived"].mean(),
        y_va["Survived"].mean(),
    ))
    model = lgb.LGBMClassifier(**params)
    model.fit(x_tr,
              y_tr,
              eval_set = [(x_tr, y_tr),(x_va, y_va)],
              early_stopping_rounds = 100, verbose = 100,
             )
    
    y_tr_pred = model.predict(x_tr)
    y_va_pred = model.predict(x_va)
    metric_tr = accuracy_score(y_tr, y_tr_pred)
    metric_va = accuracy_score(y_va, y_va_pred)
    print("[accuracy] tr: {:.2f}, va: {:.2f}".format(metric_tr, metric_va))
    metrics.append([nfold, metric_tr, metric_va])
    
    _imp = pd.DataFrame({"col":x_train.columns,
                         "imp":model.feature_importances_,
                         "nfold":nfold})
    imp = pd.concat([imp, _imp], axis = 0, ignore_index = True)
    
print("-" * 20, "result", "-" * 20)
metrics = np.array(metrics)
print(metrics)

print("[cv ] tr: {:.2f} +- {:.2f}, va: {:.2f} +- {:.2f}".format(
metrics[:, 1].mean(), metrics[:, 1].std(),
metrics[:, 2].mean(), metrics[:, 2].std(),
))

imp = imp.groupby("col")["imp"].agg(["mean", "std"])
imp.columns = ["imp", "imp_std"]
imp = imp.reset_index(drop = False)

print("Done.")

-------------------- 0 --------------------
(712, 2) (712, 1)
(179, 2) (179, 1)
y_train:0.384, y_tr:0.383, y_va:0.385
[100]	training's auc: 0.844961	valid_1's auc: 0.716469
[accuracy] tr: 0.72, va: 0.68
-------------------- 1 --------------------
(713, 2) (713, 1)
(178, 2) (178, 1)
y_train:0.384, y_tr:0.384, y_va:0.382
[100]	training's auc: 0.826717	valid_1's auc: 0.753008
[accuracy] tr: 0.75, va: 0.68
-------------------- 2 --------------------
(713, 2) (713, 1)
(178, 2) (178, 1)
y_train:0.384, y_tr:0.384, y_va:0.382
[100]	training's auc: 0.839483	valid_1's auc: 0.732687
[200]	training's auc: 0.849542	valid_1's auc: 0.737233
[accuracy] tr: 0.77, va: 0.69
-------------------- 3 --------------------
(713, 2) (713, 1)
(178, 2) (178, 1)
y_train:0.384, y_tr:0.384, y_va:0.382
[100]	training's auc: 0.831826	valid_1's auc: 0.752941
[accuracy] tr: 0.75, va: 0.69
-------------------- 4 --------------------
(713, 2) (713, 1)
(178, 2) (178, 1)
y_train:0.384, y_tr:0.383, y_va:0.388
[100]	training'

In [14]:
imp.sort_values("imp", ascending = False, ignore_index = True)

,col,imp,imp_std
0,Fare,679.390270,356.992896
1,Pclass,291.704529,138.843896


In [15]:
x_tr, x_va2, y_tr, y_va2 = train_test_split(x_train, y_train, test_size = 0.2, shuffle = True, stratify = y_train, random_state = 123)

print(x_tr.shape, y_tr.shape)
print(x_va2.shape, y_va2.shape)


(712, 2) (712, 1)
(179, 2) (179, 1)


In [16]:
x_tr1, x_va1, y_tr1, y_va1 = train_test_split(x_tr, 
                                              y_tr,
                                              test_size = 0.2,
                                              shuffle = True,
                                              stratify = y_tr,
                                              random_state = 789)
print(x_tr1.shape, y_tr1.shape)
print(x_va1.shape, y_va1.shape)

(569, 2) (569, 1)
(143, 2) (143, 1)


In [17]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.1,
    'num_leaves': 16,
    'n_estimators': 100000,
    "random_state": 123,
    "importance_type": "gain",
}

model = lgb.LGBMClassifier(**params)
model.fit(x_tr1, y_tr1, eval_set = [(x_tr1, y_tr1), (x_va1, y_va1)], early_stopping_rounds = 100, verbose = 10,)

[10]	training's auc: 0.792153	valid_1's auc: 0.72562
[20]	training's auc: 0.810025	valid_1's auc: 0.712087
[30]	training's auc: 0.822545	valid_1's auc: 0.72376
[40]	training's auc: 0.829204	valid_1's auc: 0.719421
[50]	training's auc: 0.836699	valid_1's auc: 0.720455
[60]	training's auc: 0.841808	valid_1's auc: 0.716322
[70]	training's auc: 0.84684	valid_1's auc: 0.71281
[80]	training's auc: 0.851009	valid_1's auc: 0.716529
[90]	training's auc: 0.85472	valid_1's auc: 0.720041
[100]	training's auc: 0.856354	valid_1's auc: 0.722934


LGBMClassifier(importance_type='gain', metric='auc', n_estimators=100000,
               num_leaves=16, objective='binary', random_state=123)

In [18]:
y_va1_pred = model.predict(x_va1)
y_va2_pred = model.predict(x_va2)

In [19]:
print("[検証データ] acc: {:.4f}".format(accuracy_score(y_va1, y_va1_pred)))
print("[ベースライン検証データ] acc: {:.4f}".format(accuracy_score(y_va2, y_va2_pred)))

[検証データ] acc: 0.7133
[ベースライン検証データ] acc: 0.7095


In [20]:
print("検証用データ")
print(confusion_matrix(y_va1, y_va1_pred))
print(confusion_matrix(y_va1, y_va1_pred, normalize = "all"))
print("ベースライン検証用データ")
print(confusion_matrix(y_va2, y_va2_pred))
print(confusion_matrix(y_va2, y_va2_pred, normalize = "all"))

検証用データ
[[76 12]
 [29 26]]
[[0.53146853 0.08391608]
 [0.2027972  0.18181818]]
ベースライン検証用データ
[[92 18]
 [34 35]]
[[0.51396648 0.10055866]
 [0.18994413 0.19553073]]


In [21]:
#予測値の確率算出
y_va1_pred_prob = model.predict_proba(x_va1)[:, 1]
y_va2_pred_prob = model.predict_proba(x_va2)[:, 1]
#確率値をヒストグラムで可視化
fig = plt.figure(figsize = (10, 8))
#検証データ
fig.add_subplot(2, 1, 1)
plt.title("検証データ")
plt.hist(y_va1_pred_prob[np.array(y_va1).reshape(-1) == 1], bins = 10, alpha = 0.5, label = "1")
plt.hist(y_va1_pred_prob[np.array(y_va1).reshape(-1) == 0], bins = 10, alpha = 0.5, label = "0")
plt.grid()
plt.legend()
#ベースライン検証用データ
fig.add_subplot(2, 1, 2)
plt.title("ベースライン検証用データ")
plt.hist(y_va2_pred_prob[np.array(y_va2).reshape(-1) == 1], bins = 10, alpha = 0.5, label = "1")
plt.hist(y_va2_pred_prob[np.array(y_va2).reshape(-1) == 0], bins = 10, alpha = 0.5, label = "0")
plt.grid()
plt.legend()


In [22]:
#推論用データセットの作成
df_test = pd.read_csv("../input/titanic/test.csv")
x_test = df_test[["Pclass", "Fare"]]
id_test = df_test[["PassengerId"]]

In [23]:
#学習モデルによる推論
y_test_pred = model.predict(x_test)

In [24]:
#提出用ファイルの作成
df_submit = pd.DataFrame({"PassengerId":id_test["PassengerId"], "Survived":y_test_pred})
display(df_submit.head(5))
df_submit.to_csv("submission_baseline.csv", index = None)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [25]:
#スクリプト5－1
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'learning_rate': 0.1,
    'num_leaves': 16,
    'n_estimators': 100000,
    "random_state": 123,
    "importance_type": "gain",
}

def train_cv(input_x,input_y,input_id,params,n_splits = 5,):
    metrics = []
    imp = pd.DataFrame()
    
    cv = list(StratifiedKFold(n_splits = n_splits, shuffle = True, random_state = 123).split(input_x, input_y))
    
    for nfold in np.arange(n_splits):
        print("-" * 20, nfold, "-" * 20)
        idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
        x_tr, y_tr = input_x.loc[idx_tr, :], input_y.loc[idx_tr, :]
        x_va, y_va = input_x.loc[idx_va, :], input_y.loc[idx_va, :]
        print(x_tr.shape, y_tr.shape)
        print(x_va.shape, y_va.shape)
        print("y_train:{:.3f}, y_tr:{:.3f}, y_va:{:.3f}".format(input_y["Survived"].mean(),y_tr["Survived"].mean(),y_va["Survived"].mean(),))
        
        model = lgb.LGBMClassifier(**params)
        model.fit(x_tr,y_tr,eval_set = [(x_tr, y_tr), (x_va, y_va)],early_stopping_rounds = 100,verbose = 100,)
        y_tr_pred = model.predict(x_tr)
        y_va_pred = model.predict(x_va)
        metric_tr = accuracy_score(y_tr, y_tr_pred)
        metric_va = accuracy_score(y_va, y_va_pred)
        print("[accuracy] tr: {:.2f}, va: {:.2f}".format(metric_tr, metric_va))
        metrics.append([nfold, metric_tr, metric_va])
        
        _imp = pd.DataFrame({"col":input_x.columns, "imp":model.feature_importances_,
                            "nfold": nfold})
        imp = pd.concat([imp, _imp], axis = 0, ignore_index = True)
        
    print("-" * 20, "result", "-" * 20)
    metrics = np.array(metrics)
    print(metrics)
    
    print("[cv ] tr: {:.2f} +- {:.2f}, va: {:.2f} +- {:.2f}".format(
        metrics[:, 1].mean(), metrics[:, 1].std(),
        metrics[:, 2].mean(), metrics[:, 2].std(),
    ))
    
    imp = imp.groupby("col")["imp"].agg(["mean", "std"])
    imp.columns = ["imp", "imp_std"]
    imp = imp.reset_index(drop = False)
    
    print("Done.")
    
    return imp, metrics
    

In [26]:
#スクリプト5－2
imp, metrics = train_cv(x_train, y_train, id_train, params, n_splits = 5)

-------------------- 0 --------------------
(712, 2) (712, 1)
(179, 2) (179, 1)
y_train:0.384, y_tr:0.383, y_va:0.385
[100]	training's binary_logloss: 0.470281	valid_1's binary_logloss: 0.605425
[accuracy] tr: 0.75, va: 0.70
-------------------- 1 --------------------
(713, 2) (713, 1)
(178, 2) (178, 1)
y_train:0.384, y_tr:0.384, y_va:0.382
[100]	training's binary_logloss: 0.492136	valid_1's binary_logloss: 0.564637
[200]	training's binary_logloss: 0.476557	valid_1's binary_logloss: 0.567317
[accuracy] tr: 0.76, va: 0.67
-------------------- 2 --------------------
(713, 2) (713, 1)
(178, 2) (178, 1)
y_train:0.384, y_tr:0.384, y_va:0.382
[100]	training's binary_logloss: 0.478738	valid_1's binary_logloss: 0.612512
[accuracy] tr: 0.74, va: 0.67
-------------------- 3 --------------------
(713, 2) (713, 1)
(178, 2) (178, 1)
y_train:0.384, y_tr:0.384, y_va:0.382
[100]	training's binary_logloss: 0.490942	valid_1's binary_logloss: 0.562039
[200]	training's binary_logloss: 0.47477	valid_1's bi